In [1]:
import pandas as pd
import geopandas as gpd

import numpy as np

import datetime
from datetime import datetime, timedelta


In [214]:
shp_cosecha_path = r'C:/Documents/OneDrive - Ingenio Azucarero Guabira S.A/INFORMACION GENERAL/CATASTRO/NUEVO/cosecha2024.shp'

In [215]:
gdf_cosecha = gpd.read_file(shp_cosecha_path)
area_cargado = gdf_cosecha['area'].sum()

In [216]:
gdf_cosecha['idd'] = gdf_cosecha.apply(lambda row: f"{row['unidad_01']}|{row['unidad_03']}|{row['unidad_05']}", axis=1)

In [217]:
gdf_cosecha['area'] = gdf_cosecha.geometry.area / 10000

In [218]:
area_recalculado = gdf_cosecha['area'].sum()

In [219]:
gdf_disolve = gdf_cosecha.dissolve(by='idd')
gdf_disolve['area'] = gdf_disolve.geometry.area / 10000
area_disolve = gdf_disolve['area'].sum()

In [220]:
estados = set(gdf_cosecha['estado'])

In [221]:
pivot_estados = pd.pivot_table(gdf_cosecha, values='area', index='estado', aggfunc='sum')

In [222]:
pivot_estados

,area
estado,
cosechado,2200.580990
sincosecha,51738.509301


In [223]:
cosechado = pivot_estados['area']['cosechado']
sincosecha = pivot_estados['area']['sincosecha']

In [224]:
estado_datos = {
    'Items' : ['Area cargado', 'Area recalculado', 'Area disolve', 'Estados', 'Cosechado', 'Sin cosecha', 'Suma estados'],
    'Datos' : [round(area_cargado, 2), round(area_recalculado, 2), round(area_disolve, 2), estados, round(cosechado, 2), round(sincosecha, 2), round(cosechado + sincosecha, 2)]
}
pd.DataFrame(estado_datos)

,Items,Datos
0,Area cargado,53938.92
1,Area recalculado,53939.09
2,Area disolve,53939.09
3,Estados,"{cosechado, sincosecha}"
4,Cosechado,2200.58
5,Sin cosecha,51738.51
6,Suma estados,53939.09


In [225]:
pivot_idd = pd.pivot_table(gdf_cosecha, index='idd', columns='estado', values='area', aggfunc='sum')

In [226]:
pivot_idd.columns

Index(['cosechado', 'sincosecha'], dtype='object', name='estado')

In [227]:
df_idd = pivot_idd.reset_index()

In [228]:
df_idd['cosechado'] = df_idd['cosechado'].fillna(0)
df_idd['sincosecha'] = df_idd['sincosecha'].fillna(0)

In [229]:
df_idd_select = df_idd[df_idd['cosechado']!=0]

In [230]:
df_idd_select = df_idd_select.copy()

In [231]:
df_idd_select.loc[:, 'total'] = df_idd_select['cosechado'] + df_idd_select['sincosecha']

In [232]:
df_idd_select.loc[:, 'porcen'] = df_idd_select['cosechado'] / df_idd_select['total']

In [233]:
df_idd_interes = df_idd_select[df_idd_select['porcen'] > 0.9][['idd', 'porcen']]

In [234]:
df_idd_interes

estado,idd,porcen
520,1115|20185|L8,1.0
690,114|11860|L1.4,1.0
691,114|11860|L1.5.1,1.0
717,1151|5836|L15_21,1.0
718,1151|5836|L22,1.0
...,...,...
10421,979|11158|L5,1.0
10422,979|11158|L6,1.0
10489,98|2662|L4,1.0
10491,98|2662|L6,1.0


In [235]:
gdf_disolve = gdf_disolve.reset_index()

In [236]:
df_merged = pd.merge(gdf_disolve, df_idd_interes, on='idd', how='left')

In [237]:
df_merged.columns

Index(['idd', 'geometry', 'id', 'unidad_01', 'unidad_02', 'unidad_03',
       'unidad_04', 'unidad_05', 'variedad', 'soca', 'zona', 'textura',
       'cultivo', 'zafra', 'financia', 'fs', 'fc', 'area', 'estado', 'fci',
       'tipoc', 'porcen'],
      dtype='object')

In [238]:
df_merged['area'].sum()

53939.09026723008

In [239]:
df_merged['porcen'] = df_merged['porcen'].fillna(0)

In [302]:
df_final = df_merged[df_merged['porcen']!=0]

In [310]:
df_final.columns

Index(['idd', 'geometry', 'id', 'unidad_01', 'unidad_02', 'unidad_03',
       'unidad_04', 'unidad_05', 'variedad', 'soca', 'zona', 'textura',
       'cultivo', 'zafra', 'financia', 'fs', 'fc', 'area', 'estado', 'fci',
       'tipoc', 'porcen', 'dias'],
      dtype='object')

In [304]:
df_final = df_final.copy()

In [305]:
df_final['fc'] = pd.to_datetime(df_final['fc'])
df_final['fs'] = pd.to_datetime(df_final['fs'])

In [306]:
fecha_actual = datetime.now()

In [307]:
df_final['dias'] = (fecha_actual - df_final['fc']).dt.days

In [308]:
df_final[df_final['dias'] > 10]

,idd,geometry,id,unidad_01,unidad_02,unidad_03,unidad_04,unidad_05,variedad,soca,...,zafra,financia,fs,fc,area,estado,fci,tipoc,porcen,dias
721,1151|5836|L25,"POLYGON ((518351.778 8091760.233, 518322.224 8...",000011510000L25,1151,ESTANCIA GUADALUPE--JEREZ HUGO,5836,INTEGRACION PRODUCTIVA SRL.,L25,UCG9020,4,...,2024,IAG,2019-07-25,2024-05-16,9.223266,sincosecha,0,ver,0.999992,14
959,119|7077|L1.1,"POLYGON ((491304.304 8109240.956, 491304.304 8...",00001190000L1.1,119,CAIMANES VILLA COPACABANA--CKOLO,7077,CKOLO ESCALERA CARLOS,L1.1,UCG9020,8,...,2024,None,2015-04-20,2024-05-16,1.508501,sincosecha,0,ver,1.000000,14
1384,1294|8017|L15.1,"MULTIPOLYGON (((496549.654 8094174.502, 496549...",000012940000L15.1,1294,LA FLORIDA--RIBERA ROLANDO,8017,RIBERA MERCADO ROLANDO,L15.1,UCG9020,0,...,2024,IAG,2023-05-19,2024-05-16,3.245415,sincosecha,0,ver,1.000000,14
1713,1351|7572|L1,"POLYGON ((479397.041 8096190.928, 479394.689 8...",000013510000L1,1351,LA BELLEZA--LAVERAN ZENON,7572,LAVERAN VACA ZENON,L1,RBB7726,10,...,2024,None,2013-04-19,2024-05-16,3.026443,sincosecha,0,None,1.000000,14
1764,1368|8186|L1,"POLYGON ((489189.729 8101317.834, 489187.584 8...",000013680000L1,1368,HERMANOS MERCADO--MERCADO EDUARDO,8186,MERCADO AGUILERA EDUARDO,L1,UCG9020,6,...,2024,None,2017-04-19,2024-05-16,3.613073,sincosecha,0,ver,1.000000,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8877,74|2816|C1,"POLYGON ((481302.770 8080479.566, 481238.756 8...",0000740000C1,74,LOS ARROYOS--DONOSO,2816,DONOSO BLANCO ALBERTO,C1,UCG9020,8,...,2024,None,2015-05-12,2024-05-16,7.012051,sincosecha,0,ver,1.000000,14
8879,74|2816|C3,"POLYGON ((481615.472 8079541.295, 481614.309 8...",0000740000C3,74,LOS ARROYOS--DONOSO,2816,DONOSO BLANCO ALBERTO,C3,SP835073,10,...,2024,None,2013-04-15,2024-05-16,10.134778,sincosecha,0,ver,1.000000,14
9303,809|42047|L10,"POLYGON ((483705.780 8091547.831, 483741.382 8...",00008090000L10,809,PROP.CINDA MONTAÑO,42047,TORREJON MONTAÑO ERICK FERNANDO,L10,RB2,0,...,2024,IAG,2023-05-10,2024-05-16,1.064180,cosechado,1,None,1.000000,14
9317,809|8816|L7,"POLYGON ((483509.266 8091652.062, 483503.531 8...",00008090000L7,809,PROP.CINDA MONTAÑO,8816,MONTANO GUTIERREZ CINDA,L7,RBB7726,11,...,2024,None,2012-04-20,2024-05-16,2.375828,sincosecha,0,ver,1.000000,14
